# Team Name : Hadoop Heros 


### Importing libraries and creating spark context 

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
#datawarehouse_location points to the default location for managed databases and tables
from os.path import abspath
warehouse_location = abspath('spark-warehouse')
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("ISM6562 Spark Project") \
        .enableHiveSupport() \
        .getOrCreate()

spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

23/11/10 14:36:17 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 10.21.10.196 instead (on interface eth0)
23/11/10 14:36:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/10 14:36:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark 

## Load data to spark dataframe 

In [ ]:
trips = spark.read.csv('data/yellow_tripdata_2022-02.csv' , header = True , inferSchema=True)

# display the first 10 rows of the dataframe 
trips.show(10)

In [ ]:
trips.printSchema()

## Data-preprocessing

Create a database and table in spark 

In [ ]:
trips.createOrReplaceTempView("trips_tmp_view")

In [ ]:
df= spark.sql("SELECT * FROM trips_tmp_view")
df.show(10)

In [ ]:
type(trips)

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS trips_db;")

In [ ]:
trips.write.mode("overwrite").saveAsTable("trips_db.trips")

Listing tables in the created database

In [ ]:
spark.catalog.listTables('trips_db')

In [ ]:
df = spark.sql("SELECT * FROM trips_db.trips")
df.show(10)

In [ ]:
df = spark.sql(" describe trips_db.trips")
df.show()

# Data Exploration using PySpark SQL

### Number of Trips by VendorID 

In [ ]:
vendor1_trips = spark.sql("""SELECT COUNT(VendorID) as vendorID1 
    FROM trips_db.trips 
    WHERE VendorID=1""")

vendor1_trips_1=vendor1_trips.first().vendorID1

print("Number of trips taken by Vendor 1:", vendor1_trips_1)

In [ ]:
vendor2_trips = spark.sql("""SELECT COUNT(VendorID) as vendorID2 
    FROM trips_db.trips 
    WHERE VendorID=2""")

vendor2_trips_2=vendor2_trips.first().vendorID2

print("Number of trips taken by Vendor 2:", vendor2_trips_2)

In [ ]:
import matplotlib.pyplot as plt

# Convert the result to Pandas for easy plotting
vendor1_count = vendor1_trips.toPandas()['vendorID1'][0]
vendor2_count = vendor2_trips.toPandas()['vendorID2'][0]

# Plotting the bar chart
vendors = ['VendorID 1', 'VendorID 2']
counts = [vendor1_count, vendor2_count]

plt.bar(vendors, counts, color=['blue', 'green'])
plt.title('Number of Trips for VendorID 1 and VendorID 2')
plt.xlabel('VendorID')
plt.ylabel('Number of Trips')
plt.show()


Analysis:

Vendor1 and Vendor2 had 319007, 724578 trips each in total
Vendor2 took more than twice the number of trips of Vendor1

### Number of trips by pickup and dropff location 

In [ ]:
# count of each pickup location 

pickup_trips = spark.sql("""SELECT PULocationID, COUNT(*) AS pickup_count 
    FROM trips_db.trips 
    GROUP BY PULocationID
    ORDER BY pickup_count DESC""")

# Convert the DataFrame to a Pandas DataFrame
pickup_trips_pandas = pickup_trips.toPandas()

# Print the Pandas DataFrame
print(pickup_trips_pandas.head(10))


In [ ]:
# count of each dropoff location 

dropoff_trips = spark.sql("""SELECT DOLocationID, COUNT(*) AS dropoff_count 
    FROM trips_db.trips
    GROUP BY DOLocationID 
    ORDER BY dropoff_count DESC""")

# Convert the DataFrame to a Pandas DataFrame
dropoff_trips_pandas = dropoff_trips.toPandas()

# Print the Pandas DataFrame
print(dropoff_trips_pandas.head(10))


Analysis:
The most frequented pick up and drop off location are  Taxi Zones 237 & 236

### Which vendor provides a better service? 
(Assumption : Tipping being a metric for good service)

In [ ]:
avg_tip_per_vendor = spark.sql("""SELECT VendorID, ROUND(AVG(tip_amount),2) AS avg_tip 
    FROM trips_db.trips 
    GROUP BY VendorID 
    ORDER BY avg_tip DESC """)

avg_tip_per_vendor_pandas=avg_tip_per_vendor.toPandas()
print(avg_tip_per_vendor_pandas)

Analysis:

The avg_tip difference between both vendors is minimal.
Based on this result, it is difficult to draw a conclusion on service quality.

### What is the average fare for the trips during rush hours with extra charges? 

In [ ]:
avg_fare_amount_rush_hour = spark.sql("""SELECT ROUND(AVG(fare_amount),3) as avg_fare_amount 
    FROM trips_db.trips 
    WHERE extra > 0""")

# Extract the average fare amount value
average_fare = avg_fare_amount_rush_hour.first()[0]

# Print the statement
print(f"Average Fare  for trips during rush hours is: ${average_fare:.2f}")

### What is the average total amount collected for trips with more than 2 passengers ? 

In [ ]:
avg_total_for_large_groups = spark.sql("""SELECT ROUND(AVG(total_amount),2) as total_fare 
    FROM trips_db.trips  
    WHERE passenger_count > 2""")

# Extract the average fare amount value
average_total = avg_total_for_large_groups.first()[0]

# Print the statement
print(f"Average total amount for trips with more than 2 passengers: ${average_total:.2f}")

### What is the avergare trip distance for each passenger count? 

In [ ]:
avg_distance_per_passenger = spark.sql("""SELECT passenger_count, ROUND(AVG(trip_distance),2) as avg_distance 
    FROM trips_db.trips 
    GROUP BY passenger_count
    ORDER BY passenger_count""")

avg_distance_per_passenger_pandas = avg_distance_per_passenger.toPandas()
print(avg_distance_per_passenger_pandas)

In [ ]:
import matplotlib.pyplot as plt

plt.bar(avg_distance_per_passenger_pandas['passenger_count'], avg_distance_per_passenger_pandas['avg_distance'], color='lightgreen', alpha=0.7)
plt.xlabel('Passenger Count')
plt.ylabel('Average Distance')
plt.title('Average Distance per Passenger Count')
plt.show() 

### What is the relation between trip distance and fare amount ? 

In [ ]:
trips_pd = trips.toPandas()

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(trips_pd['trip_distance'], trips_pd['fare_amount'], alpha=0.5)
plt.title('Trip distance Vs Fare amount')
plt.xlabel('Trip Distance (miles)')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.show()

Analysis

From the above scatter plot, general trend seems to increase in fare amount for every additional mile travelled . 
Negative fares might represent refund amount for cancelled trips.
Zero fare amount for distance travelled may represnt promotional offers for the ride. 
Fare amounts for zero distance travelled might be cancellation of trips after late cancellation or minimum fare charged regardless of distance travelled. 

## Understanding the data structure and Data Cleaning

In [ ]:
# Retreving column names
trips.columns

In [ ]:
# number of rows
num_rows = trips.count()

#  number of columns
num_columns = len(trips.columns)

print("Number of rows: {}".format(num_rows))
print("Number of columns: {}".format(num_columns))


In [ ]:
trips.printSchema()

## Extracting required columns for analysis.

In [ ]:
selected_columns = ['VendorID', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'extra', 'airport_fee']

In [ ]:
selected_data = trips.select(selected_columns)

In [ ]:
# Drop rowns with any null values 
selected_data = selected_data.dropna() 

In [ ]:
selected_data.printSchema()

In [ ]:
# Converting VendorID, PULocationID, DOLocationID, airport_fee to categorical variables

In [ ]:
from pyspark.sql.types import StringType

selected_data = selected_data.withColumn("VendorID",selected_data.VendorID.cast(StringType()))
selected_data = selected_data.withColumn("PULocationID",selected_data.PULocationID.cast(StringType()))
selected_data = selected_data.withColumn("DOLocationID",selected_data.DOLocationID.cast(StringType()))
selected_data = selected_data.withColumn("airport_fee",selected_data.airport_fee.cast(StringType()))

In [ ]:
selected_data.printSchema()

Now that we have our data ready, let's do a train test split (70/30).

In [ ]:
train_data,test_data=selected_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.feature import StringIndexer
# Use StringIndexer to convert the categorical columns to hold numerical data
 
VendorID_indexer = StringIndexer(inputCol='VendorID',outputCol='VendorID_index',handleInvalid='keep')
PULocationID_indexer = StringIndexer(inputCol='PULocationID',outputCol='PULocationID_index',handleInvalid='keep')
DOLocationID_indexer = StringIndexer(inputCol='DOLocationID',outputCol='DOLocationID_index',handleInvalid='keep')
airport_fee_indexer = StringIndexer(inputCol='airport_fee',outputCol='airport_fee_index',handleInvalid='keep')


In [ ]:
from pyspark.ml.feature import VectorAssembler
# Vector assembler is used to create a vector of input features
 
assembler = VectorAssembler(
    inputCols=[
        'VendorID_index',
        'PULocationID_index',
        'DOLocationID_index',
        'airport_fee_index',
        'extra',
        'trip_distance',
        'passenger_count',
    ],
    outputCol="features"
)

In [ ]:
from pyspark.ml import Pipeline

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

 
pipe = Pipeline(stages=[
    VendorID_indexer,
    PULocationID_indexer,
    DOLocationID_indexer,
    airport_fee_indexer,
    assembler
    ]
)

In [ ]:
fitted_pipe=pipe.fit(train_data)

In [ ]:
train_data=fitted_pipe.transform(train_data)
train_data.show(5)

In [ ]:
test_data=fitted_pipe.transform(test_data)
test_data.show()

In [ ]:
from pyspark.ml.regression import LinearRegression

lr_model = LinearRegression(labelCol='fare_amount')
fit_model = lr_model.fit(train_data.select(['features','fare_amount']))

In [ ]:
results = fit_model.transform(test_data)
results.show(10)

In [ ]:
results.select(['fare_amount','prediction']).show()

## Evaluate the peformance of the Linear Regression Model

In [ ]:
test_results_lr = fit_model.evaluate(test_data)

In [ ]:
test_results_lr.residuals.show()

In [ ]:
rmse_linear=test_results_lr.rootMeanSquaredError
r2_linear=test_results_lr.r2

print(f"{'RMSE LinearReg:':7s} {test_results_lr.rootMeanSquaredError:>7.3f}")
print(f"{'MAE LinearReg:':7s} {test_results_lr.meanAbsoluteError:>7.3f}")
print(f"{'MSE LinearReg:':7s} {test_results_lr.meanSquaredError:>7.3f}")
print(f"{'R2 LinearReg:':7s} {test_results_lr.r2:>7.3f}")

## Decision Tree 

In [ ]:
from pyspark.ml.regression import  DecisionTreeRegressor

In [ ]:
dt_model = DecisionTreeRegressor(labelCol='fare_amount',maxBins=5000)
fit_model = dt_model.fit(train_data.select(['features','fare_amount']))

In [ ]:
pipe = Pipeline(
    stages=[
        VendorID_indexer,
        PULocationID_indexer,
        DOLocationID_indexer,
        airport_fee_indexer,
        assembler,
        dt_model
    ]
)

In [ ]:
results_dt = fit_model.transform(test_data)

In [ ]:
results_dt.select(['fare_amount','prediction']).show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
# Define the evaluator
evaluator = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction")

# Calculate the metrics
rmse_dtree = evaluator.evaluate(results_dt, {evaluator.metricName: "rmse"})
mae_dtree = evaluator.evaluate(results_dt, {evaluator.metricName: "mae"})
mse_dtree = evaluator.evaluate(results_dt, {evaluator.metricName: "mse"})
r2_dtree = evaluator.evaluate(results_dt, {evaluator.metricName: "r2"})

In [ ]:
# Print the metrics
print(f"{'RMSE Dtree:':7s} {rmse_dtree:>7.3f}")
print(f"{'MAE Dtree:':7s} {mae_dtree:>7.3f}")
print(f"{'MSE Dtree:':7s} {mse_dtree:>7.3f}")
print(f"{'R2 Dtree:':7s} {r2_dtree:>7.3f}")

## RANDOM FOREST

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

# Random Forest model
rf_model = RandomForestRegressor(labelCol='fare_amount', featuresCol='features', numTrees=100, maxBins=260)
fit_model = rf_model.fit(train_data.select(['features','fare_amount']))

In [ ]:
# Make predictions on the test data

results_rf = fit_model.transform(test_data)
results_rf.select(['fare_amount', 'prediction']).show()

### Evaluating the model

In [ ]:
# Evaluate the Random Forest model
evaluator = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction")

rmse_rf = evaluator.evaluate(results_rf, {evaluator.metricName: "rmse"})
mae_rf = evaluator.evaluate(results_rf, {evaluator.metricName: "mae"})
mse_rf = evaluator.evaluate(results_rf, {evaluator.metricName: "mse"})
r2_rf = evaluator.evaluate(results_rf, {evaluator.metricName: "r2"})

In [ ]:
print(f"{'RMSE (Random Forest):':22s} {rmse_rf:>7.3f}")
print(f"{'MAE (Random Forest):':22s} {mae_rf:>7.3f}")
print(f"{'MSE (Random Forest):':22s} {mse_rf:>7.3f}")
print(f"{'R2 (Random Forest):':22s} {r2_rf:>7.3f}")

# RESULTS

We built 3 models for predicting the fare price based on the input features like VendorID, Pickup location and drop off location ID's, airport fee, extra charges, total trip distance, and passenger count.

Upon evaluating metrics, these are the results:


In [ ]:
print(f"{'Linear regression Model'}")
print(f"{'RMSE :':22s}{rmse_linear:>7.3f}")
print(f"{'R-squared error :':22s}{r2_linear:>7.3f}")

print("*****************************")
print(f"{'Decision Tree Model'}")
print(f"{'RMSE :':22s}{rmse_dtree:>7.3f}")
print(f"{'R-squared error:':22s}{r2_dtree:>7.3f}")

print("*****************************")
print(f"{'Random Forest Model'}")
print(f"{'RMSE :':22s}{rmse_rf:>7.3f}")
print(f"{'R-squared error :':22s}{r2_rf:>7.3f}")

spark.stop()